In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

<img src="CNN.jpeg" width=600 height=600 />

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyper-parameters 
num_epochs = 5
batch_size = 4
learning_rate = 0.001

In [4]:
#dataset has PILImage images of range [0, 1]
#we transform them to Tensors od normalised range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                             download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                             shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#implement conv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

In [6]:
#loss and optimizer
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
#training
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [2000/12500], Loss: 2.3143
Epoch [1/5], Step [4000/12500], Loss: 2.2695
Epoch [1/5], Step [6000/12500], Loss: 2.2965
Epoch [1/5], Step [8000/12500], Loss: 2.2913
Epoch [1/5], Step [10000/12500], Loss: 2.2616
Epoch [1/5], Step [12000/12500], Loss: 2.3239
Epoch [2/5], Step [2000/12500], Loss: 2.1821
Epoch [2/5], Step [4000/12500], Loss: 1.9384
Epoch [2/5], Step [6000/12500], Loss: 2.1050
Epoch [2/5], Step [8000/12500], Loss: 2.3301
Epoch [2/5], Step [10000/12500], Loss: 1.8866
Epoch [2/5], Step [12000/12500], Loss: 2.0290
Epoch [3/5], Step [2000/12500], Loss: 1.1342
Epoch [3/5], Step [4000/12500], Loss: 1.1940
Epoch [3/5], Step [6000/12500], Loss: 1.7051
Epoch [3/5], Step [8000/12500], Loss: 1.7827
Epoch [3/5], Step [10000/12500], Loss: 2.0902
Epoch [3/5], Step [12000/12500], Loss: 1.0950
Epoch [4/5], Step [2000/12500], Loss: 1.3925
Epoch [4/5], Step [4000/12500], Loss: 1.2161
Epoch [4/5], Step [6000/12500], Loss: 2.4988
Epoch [4/5], Step [8000/12500], Loss: 2.2571
Epoc

In [8]:
#Testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 48.17 %
Accuracy of plane: 62.0 %
Accuracy of car: 66.5 %
Accuracy of bird: 32.6 %
Accuracy of cat: 18.8 %
Accuracy of deer: 25.9 %
Accuracy of dog: 54.5 %
Accuracy of frog: 74.1 %
Accuracy of horse: 49.6 %
Accuracy of ship: 40.1 %
Accuracy of truck: 57.6 %
